<a href="https://colab.research.google.com/github/Nachikaet/River-water-level--LSTM/blob/main/Vision%20transformer%20river.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Full corrected script — uses your filenames & credentials and re-authenticates each loop.

!pip install rasterio geopandas shapely transformers torch torchvision tqdm requests --quiet

import os
import glob
import time
import json
import requests
import urllib.parse
import numpy as np
import pandas as pd
import torch
from zipfile import ZipFile
from tqdm import tqdm
import rasterio
import cv2
from shapely.geometry import Polygon
from transformers import AutoFeatureExtractor, AutoModel

# -------------------------
# Your Copernicus credentials (as provided by you)
# -------------------------
CATALOG_URL = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
AUTH_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
USERNAME = "nachikaet10l20@gmail.com"
PASSWORD = "Sigmaskibidi2005*"

# -------------------------
# Drive paths (as you used)
# -------------------------
DRIVE_CSV_IN = "/content/drive/My Drive/ML/finalLSTM.csv"
DRIVE_OUTPUT = "/content/drive/My Drive/ML/extracted_features_with_dates.csv"

# local working dir for downloads/extract
WORK_DIR = "data"
os.makedirs(WORK_DIR, exist_ok=True)

# -------------------------
# Model / extractor (same as your original)
# -------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = AutoModel.from_pretrained("google/vit-base-patch16-224-in21k")
model.eval().to(device)

# -------------------------
# Authentication helper (same flow you used)
# -------------------------
def authenticate():
    data = {
        "client_id": "cdse-public",
        "grant_type": "password",
        "username": USERNAME,
        "password": PASSWORD
    }
    r = requests.post(AUTH_URL, data=data, timeout=30)
    r.raise_for_status()
    tok = r.json().get("access_token")
    if not tok:
        raise RuntimeError("Authentication returned no access_token")
    return tok

# -------------------------
# Query helper (builds url-encoded OData filter)
# -------------------------
def query_catalog(filter_expr, token, timeout=60):
    q = {
        "$filter": filter_expr,
        "$top": "1",
        "$orderby": "ContentDate/Start desc"
    }
    # build URL with encoded query string
    query_str = "&".join(f"{k}={urllib.parse.quote(v, safe='')}" for k, v in q.items())
    url = f"{CATALOG_URL}?{query_str}"
    headers = {"Authorization": f"Bearer {token}"}
    resp = requests.get(url, headers=headers, timeout=timeout)
    # caller handles 401/403 by reauthenticating (we reauth every loop anyway)
    resp.raise_for_status()
    return resp.json()

# -------------------------
# Helper: clean extracted files
# -------------------------
def cleanup_data_dir():
    import shutil
    for p in glob.glob(os.path.join(WORK_DIR, "*")):
        try:
            if os.path.isdir(p):
                shutil.rmtree(p)
            else:
                os.remove(p)
        except Exception:
            pass

# -------------------------
# Load and prepare monthly list (keeps your original logic, fixed for Period/Timestamp)
# -------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

df = pd.read_csv(DRIVE_CSV_IN)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
df = df[df['date'] >= "2015-06-23"]

monthly_df = df.groupby(df['date'].dt.to_period('M'), as_index=False).first().rename(columns={'date': 'month_period'})
monthly_df['date'] = monthly_df['month_period'].apply(lambda x: x.to_timestamp() if hasattr(x, "to_timestamp") else pd.Timestamp(x))
monthly_df.drop(columns=['month_period'], inplace=True)

# -------------------------
# Main loop: re-auth each iteration (as you requested)
# -------------------------
results = []
total = len(monthly_df)
for idx, row in tqdm(monthly_df.iterrows(), total=total, desc="Months"):
    date = row['date']
    # re-authenticate every iteration (explicitly)
    try:
        token = authenticate()
    except Exception as e:
        print(f"Auth failed at {date}: {e}")
        # append NaN row and continue
        nan_feat = [np.nan] * 768
        rec = {"date": date}
        rec.update({f"feat_{i}": float("nan") for i in range(len(nan_feat))})
        results.append(rec)
        continue

    start_date = (date - pd.Timedelta(days=7)).strftime("%Y-%m-%dT00:00:00.000Z")
    end_date   = (date + pd.Timedelta(days=7)).strftime("%Y-%m-%dT23:59:59.000Z")

    filter_expr = (
        "Collection/Name eq 'SENTINEL-2' and "
        "Attributes/OData.CSC.StringAttribute/any(a: a/Name eq 'productType' and a/Value eq 'S2MSI2A') and "
        "Attributes/OData.CSC.DoubleAttribute/any(a: a/Name eq 'cloudCover' and a/Value lt 15) and "
        f"ContentDate/Start gt {start_date} and ContentDate/Start lt {end_date}"
    )

    feat = np.full(768, np.nan)  # default

    try:
        resp_json = query_catalog(filter_expr, token)
        items = resp_json.get("value", [])
        if not items:
            # No product for this month: keep NaN vector
            print(f"No Sentinel-2 item found for {date}")
            rec = {"date": date}
            rec.update({f"feat_{i}": float("nan") for i in range(len(feat))})
            results.append(rec)
            # polite delay
            time.sleep(1.0)
            cleanup_data_dir()
            continue

        product = items[0]
        pid = product["Id"]
        pname = product["Name"]
        zip_path = os.path.join(WORK_DIR, f"{pname}.zip")

        # Download product zip (reauth handled above)
        dl_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({pid})/$value"
        with requests.get(dl_url, headers={"Authorization": f"Bearer {token}"}, stream=True, timeout=120) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

        # Extract zip to WORK_DIR
        with ZipFile(zip_path, "r") as z:
            z.extractall(WORK_DIR)

        # find SAFE directory
        safe_dirs = [d for d in os.listdir(WORK_DIR) if d.endswith(".SAFE")]
        if not safe_dirs:
            raise ValueError("No .SAFE directory found after extraction")
        safe_dir = safe_dirs[0]

        # find 10m RGB bands (B02,B03,B04) — pattern used earlier
        band_paths = glob.glob(os.path.join(WORK_DIR, safe_dir, "GRANULE", "*", "IMG_DATA", "R10m", "*B0[234]_10m.jp2"))
        band_paths.sort()
        if len(band_paths) < 3:
            raise ValueError("Incomplete RGB band files (expected 3). Found: {}".format(len(band_paths)))

        # Read bands
        bands = []
        for b in band_paths:
            with rasterio.open(b) as src:
                arr = src.read(1).astype(np.float32)
                # treat all-NaN or all-zero as invalid
                if np.all(np.isnan(arr)) or np.all(arr == 0):
                    raise ValueError(f"Invalid band (all NaN or zero): {b}")
                bands.append(arr)

        # Stack into RGB (ensure correct order: depending on file names order might be B02,B03,B04)
        # band_paths sorted should align with B02,B03,B04; we will stack as (B04,B03,B02) => R,G,B if necessary.
        # To be safe, inspect names and pick according to B0X substring:
        def band_index(path):
            basename = os.path.basename(path)
            if "B02" in basename: return 2
            if "B03" in basename: return 3
            if "B04" in basename: return 4
            return 0
        ordered = sorted(band_paths, key=band_index)
        # Now read ordered arrays
        bands = []
        for b in ordered:
            with rasterio.open(b) as src:
                arr = src.read(1).astype(np.float32)
                bands.append(arr)

        # Determine mapping to R,G,B: B04->R, B03->G, B02->B
        # Create dict mapping
        band_map = {}
        for p in ordered:
            name = os.path.basename(p)
            if "B02" in name: band_map['B02'] = p
            if "B03" in name: band_map['B03'] = p
            if "B04" in name: band_map['B04'] = p

        # read arrays in correct R,G,B order
        with rasterio.open(band_map['B04']) as sR:
            R = sR.read(1).astype(np.float32)
        with rasterio.open(band_map['B03']) as sG:
            G = sG.read(1).astype(np.float32)
        with rasterio.open(band_map['B02']) as sB:
            B = sB.read(1).astype(np.float32)

        rgb = np.stack([R, G, B], axis=-1)  # H x W x 3

        # replace NaNs -> 0
        rgb = np.nan_to_num(rgb, nan=0.0)

        # robust normalization:
        upper = np.nanpercentile(rgb, 99)
        if (upper == 0) or np.isnan(upper):
            upper = np.nanmax(rgb)
        if (upper == 0) or np.isnan(upper):
            raise ValueError("Image is all zeros / cannot normalize")

        rgb = np.clip(rgb / float(upper), 0.0, 1.0)

        # resize to 224x224 (ViT input)
        rgb_resized = cv2.resize(rgb, (224, 224), interpolation=cv2.INTER_LINEAR)

        # Convert to uint8 0-255 to avoid extractor rescale warnings (extractor expects 0-255 by default)
        img_uint8 = (np.clip(rgb_resized, 0.0, 1.0) * 255.0).round().astype(np.uint8)

        # Feed extractor / model (move tensors to device)
        inputs = extractor(images=img_uint8, return_tensors="pt", do_resize=False, do_rescale=False)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
        # feature vector
        feat = outputs.pooler_output.cpu().numpy().flatten()

        # convert to Python floats for CSV
        feat = feat.astype(float)

    except Exception as e:
        print(f"Error processing {date}: {e}")
        feat = np.full(768, np.nan)

    finally:
        # cleanup files extracted each iteration to conserve disk
        cleanup_data_dir()

    # append record with date and features
    rec = {"date": date}
    rec.update({f"feat_{i}": (float(v) if (not np.isnan(v)) else float("nan")) for i, v in enumerate(feat)})
    results.append(rec)

    # polite small delay
    time.sleep(1.0)

# Save results
out_df = pd.DataFrame(results)
out_df.to_csv(DRIVE_OUTPUT, index=False)
print("Saved features to:", DRIVE_OUTPUT)


/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Mounted at /content/drive


Months:   0%|          | 0/125 [00:00<?, ?it/s]

No Sentinel-2 item found for 2015-06-23 00:00:00


Months:  35%|███▌      | 44/125 [1:38:59<1:43:43, 76.83s/it]

Error processing 2019-02-01 00:00:00: Invalid band (all NaN or zero): data/S2B_MSIL2A_20190208T235259_N0500_R030_T57PZK_20221126T103409.SAFE/GRANULE/L2A_T57PZK_A010066_20190208T235300/IMG_DATA/R10m/T57PZK_20190208T235259_B02_10m.jp2


Months:  97%|█████████▋| 121/125 [5:13:55<11:57, 179.31s/it]